In [157]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 30)

## Chargement des données

On commence par charger le fichier qui indique les réponses des participants au questionnaire. Les réponses aux questions sont codées par des nombres, la signification des codes est fournie dans un fichier annexe de documentation. On va donc devoir jongler entre les deux.

In [158]:
df_brut = pd.read_excel("Data/Etude sur les Français et l_information/les-francais-et-l-information-arcom-2024-base-anonymisee.xlsx")
df_doc = pd.read_excel("Data/Etude sur les Français et l_information/Documentation/les-francais-et-l-information-arcom-2024-datamap.xlsx", sheet_name='TEXTS')

print("format des données brutes: ", df_brut.shape)

format des données brutes:  (3346, 1076)


## Premier nettoyage des données

Le fichier des données brutes contient des colonnes récapitulatives qui donnent une information redondante, on va supprimer celles-ci pour alléger le dataset.

In [159]:
# Suppression des colonnes "récap" dans la documentation
colonnes_recap = list(df_doc[df_doc["FR:L"].str.contains("Récap", case=False, na=False)]["NAME"])
colonnes_recap += list(df_doc[df_doc["FR:L"].str.contains("Recap", case=False, na=False)]["NAME"])
colonnes_recap += ["INT3_R7", "RSINFO_2_LR_R2", "RSINFO_2_LR_R3", "RSINFO_2_LR_R4", "COMPLOT2_R2"]

# Suppression des colonnes "récap" dans les données
colonnes_recap = [col for col in df_brut.columns if any(mot in col for mot in colonnes_recap)]
df_new = df_brut.drop(columns=colonnes_recap)

print(f"{len(colonnes_recap)} colonnes ont été supprimées. Il en reste {df_new.shape[1]}.")

537 colonnes ont été supprimées. Il en reste 539.


On va maintenant parcourir le fichier de documentation et construire un dictionnaire qui nous donne, pour chaque question sa documentation spécifique afin de faciliter les prochains programmes.

Le fichier est une table dans laquelle certaines lignes indiquent l'intitulé d'une question (la colonne "TYPE" indique alors "TITLE") et les lignes suivantes donnent alors le code correspondant aux différentes réponses (la colonne "TYPE" indique alors "LABEL").

On parcours donc le fichier et dès qu'on rencontre une ligne TITLE, on récupère toutes les lignes LABEL qui suivent jusqu'au prochain TITLE.

On obtient un dataframe pour chaque question mais ce n'est pas suffisant. En effet pour certaines questions, la documentation des codes est indiquée dans une autre question qui la précède. De plus pour certaines question, seule une partie du dataframe nous interesse car le reste indique la signification des colonnes que nous traiterons à part. Malheureusement, on ne peut que modifier ceci à la main.

In [ ]:
# Dictionnaire pour stocker les résultats
documentation = {}

# Parcours du DataFrame
current_key = None  # Clé pour le dictionnaire, c'est l'indentificateur de la question
start_index = None  # Indique l'index de début des lignes LABEL qui correspondent au réponses à une qu

for i, row in df_doc.iterrows():
    if row["TYPE"] == "TITLE":
        # Sauvegarde le sous-DataFrame précédent si un TITLE est déjà en cours
        if current_key is not None and start_index is not None:
            documentation[current_key] = df_doc.iloc[start_index:i]

        # Met à jour la clef courant et le début des lignes LABEL
        current_key = row["NAME"]
        start_index = i + 1

# Ajoute la documentation de la dernière question après le dernier TITLE
if current_key is not None and start_index is not None:
    documentation[current_key] = df_doc.iloc[start_index:]
    
# Supprime les entrées qui correspondent à des colonne récap
keys_to_remove = {"_".join(elem.split("_")[:2]) for elem in colonnes_recap}  
documentation = {key: value for key, value in documentation.items() if key not in keys_to_remove}

# Supprime quelques entrées inutiles
del documentation["NOU1_R"], documentation["NBR_NEWS1SOURCES_R1"], documentation["NBR_NEWS1SOURCES_R3"]
del documentation["RSINFO_2_LR_R2"], documentation["RSINFO_2_LR_R3"], documentation["RSINFO_2_LR_R4"],
del documentation["SOURCES1A_DOMR_R"], documentation["LEVEL_A"], documentation["RECORD"], documentation["UUID"]

noms_colonnes = {}

# Reprécise la documentation pour certaines questions
documentation["RS16_R"] = documentation["RS16_R"].loc[132:134]
documentation["INT2_R"] = documentation["INT1_R"]
documentation["MOTIV_R"] = documentation["MOTIV_R"].loc[216:219]
documentation["NEWS1_R"] = documentation["NEWS1_R"].loc[252:257]
documentation["NEWS1SOURCES_LR7_R"] = documentation["NEWS1SOURCES_LR6_R"]
documentation["NEWS1SOURCES_LR8_R"] = documentation["NEWS1SOURCES_LR6_R"]
documentation["NEWS1SOURCES_LR9_R"] = documentation["NEWS1SOURCES_LR6_R"]
documentation["DEVICEXMOMENTS_R1"] = documentation["DEVICEXMOMENTS_R1"].loc[338:345]
documentation["FORMAT3_LR2_FORMAT3_2_R"] = documentation["FORMAT3_LR1_FORMAT3_1_R"]
documentation["FORMAT3_LR3_FORMAT3_3_R"] = documentation["FORMAT3_LR1_FORMAT3_1_R"]
documentation["FORMAT2_LR3_FORMAT2_3_R"] = documentation["FORMAT2_LR1_FORMAT2_1_R"]
documentation["PAY_R"] = documentation["PAY_R"].loc[369:370]
documentation["AGREG_R"] = documentation["AGREG_R"].loc[388:390]
documentation["NOTIF_R"] = documentation["PAY_R"].loc[369:370]
documentation["RSINFO_2_LR_R"] = documentation["PAY_R"].loc[369:370]
documentation["ENGAR_R"] = documentation["ENGAR_R"].loc[473:475]
documentation["INF1_R"] = documentation["INF1_R"].loc[633:638]
documentation["INF3_R"] = documentation["INF3_R"].loc[691:694]
documentation["ATTSOURCES_R"] = documentation["ATTSOURCES_R"].loc[716:719]
documentation["OP2_R"] = documentation["OP2_R"].loc[757:760]
documentation["COMPLOT1_R"] = documentation["PAY_R"].loc[369:370]
documentation["COMPLOT2_R"] = documentation["PAY_R"].loc[369:370]
documentation["VALEURJ_R"] = documentation["INF1_R"].loc[633:638]
documentation["ATTENTESJ_R"] = documentation["ATTENTESJ_R"].loc[802:804]
documentation["DEFIANCEJ_R"] = documentation["DEFIANCEJ_R"].loc[834:837]
documentation["CONNAISSALGOA_R"] = documentation["CONNAISSALGOA_R"].loc[862:863]
documentation["OPTI_R"] = documentation["OPTI_R"].loc[904:907]
documentation["ATTA_R"] = documentation["ATTA_R"].loc[924:927]
documentation["DEM_R"] = documentation["DEM_R"].loc[948:951]
documentation["CONFINST_R"] = documentation["INF3_R"].loc[691:694]
documentation["PERS_R"] = documentation["PAY_R"].loc[369:370]
documentation["VA1_R"] = documentation["VA1_R"].loc[1009:1013]


# Affichage de la documentation
for key, sub_df in documentation.items():
    print(f"Bloc '{key}':")
    print(sub_df)
    print()


Bloc 'CIBLE':
  HEADER TEXT   NAME   TYPE  CODE  VALUE  FORMULA  BASE  GROUPID  \
4        TEXT  CIBLE  LABEL   1.0    NaN      NaN   NaN      NaN   
5        TEXT  CIBLE  LABEL   2.0    NaN      NaN   NaN      NaN   
6        TEXT  CIBLE  LABEL   3.0    NaN      NaN   NaN      NaN   

                    FR:L  
4  France métropolitaine  
5                DOM TOM  
6                DOM - 2  

Bloc 'SOURCE_BASE_R1':
   HEADER TEXT            NAME   TYPE  CODE  VALUE  FORMULA  BASE  GROUPID  \
8         TEXT  SOURCE_BASE_R1  LABEL   1.0    NaN      NaN  -1.0      1.0   
9         TEXT  SOURCE_BASE_R1  LABEL   2.0    NaN      NaN   NaN      1.0   
10        TEXT  SOURCE_BASE_R1  LABEL   3.0    NaN      NaN   NaN      1.0   

              FR:L  
8             CATI  
9   CAWI Métropole  
10  CAWI Outre-mer  

Bloc 'POIDS03':
Empty DataFrame
Columns: [HEADER TEXT, NAME, TYPE, CODE, VALUE, FORMULA, BASE, GROUPID, FR:L]
Index: []

Bloc 'INTER':
Empty DataFrame
Columns: [HEADER TEXT, NAME, TYP

On va maintenant utiliser ce dictionnaire de documentations pour transformer toutes les valeurs codées numériquement par la réponse qui correspond dans le DataFrame. Celà permettra de l'utiliser plus simplement via un outil de création de dashboard par exemple.

In [165]:
# Parcours de la doc 
for i, question in enumerate(documentation.keys()):
    
    # Colonnes concernées par cette question et documentation associée
    liste_col = [col for col in df_new.columns if col.startswith(question)]
    doc = documentation[question]
    
    # Remplacement des codes par leur signification
    for col in liste_col:
        df_new[col] = df_new[col].apply(lambda x: doc[doc["CODE"]==x]["FR:L"].iloc[0] if x in doc["CODE"].unique() else x)
    
    # Checking de l'avancement 
    print(f"\rAvancement: étape {i:03d} / {len(documentation.keys()):03d}", end='', flush=True)

# Affichage du résultat
df_new


Avancement: étape 000 / 094

C:\Users\totog\AppData\Local\Temp\ipykernel_14848\92474698.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  df_new[col] = df_new[col].apply(lambda x: doc[doc["CODE"]==x]["FR:L"].iloc[0] if x in doc["CODE"].unique() else x)


Avancement: étape 093 / 094

,RECORD,UUID,CIBLE,SOURCE_BASE_R1,POIDS03,INTER,CONTACT,RS1_R,RS2C_RECODE_AG_R,RS3_R,...,VA1_R_6,VA1_R_7,VA1_R_8,NOU1_R,PP3_R,QUIZZ1_R,QUIZZ2_R,QUIZZ3_R,QUIZZ4_R,QUESTION_OUVERTE
0,NaN,NaN,NaN,CATI,"4,0000087157244",68964.0,NaN,Un homme,60-69 ans,Retraité·e,...,Tout à fait,Tout à fait,Pas du tout,7.0,Emmanuel Macron,Je ne sais pas,David Cameron,2026,Je ne sais pas,NaN
1,NaN,NaN,NaN,CATI,"3,58505412001293",62701.0,NaN,Une femme,60-69 ans,Retraité·e,...,Tout à fait,Tout à fait,Tout à fait,6.0,Je ne suis pas inscrit sur les listes électorales,Bonne réponse,David Cameron,2026,"0,12",NaN
2,NaN,NaN,NaN,CATI,"4,0000087157244",64008.0,NaN,Une femme,70 ans et +,Retraité·e,...,Plutôt,Plutôt pas,Pas du tout,1.0,Jean Luc Mélenchon,Bonne réponse,Theresa May,Bonne réponse,Bonne réponse,NaN
3,NaN,NaN,NaN,CATI,"4,00000441186288",16039.0,NaN,Un homme,35-44 ans,"Actif, active (ayant un emploi, y compris en c...",...,Plutôt,Plutôt pas,Plutôt,6.0,J'ai voté blanc ou nul,Je ne sais pas,Bonne réponse,2026,Je ne sais pas,NaN
4,NaN,NaN,NaN,CATI,"4,0000087157244",9961.0,NaN,Une femme,70 ans et +,Retraité·e,...,Tout à fait,Tout à fait,Pas du tout,6.0,Emmanuel Macron,Bonne réponse,Bonne réponse,Bonne réponse,Bonne réponse,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,5911.0,69bwh8msjc15z946,DOM TOM,CAWI Outre-mer,"0,502029079078932",NaN,NaN,Une femme,35-44 ans,"Chômeur, chômeuse ayant déjà travaillé",...,Plutôt pas,Plutôt,Pas du tout,8.0,J'ai voté blanc ou nul,Je ne sais pas,Je ne sais pas,2026,Bonne réponse,Merci
3342,5913.0,xz15h04427pzqr0g,DOM TOM,CAWI Outre-mer,"0,587583082396885",NaN,NaN,Un homme,35-44 ans,"Chômeur, chômeuse ayant déjà travaillé",...,Plutôt,Plutôt pas,Plutôt pas,6.0,J'ai voté blanc ou nul,Je ne sais pas,David Cameron,2026,Je ne sais pas,néant
3343,5914.0,bax5h2zqga76z6q0,DOM TOM,CAWI Outre-mer,"0,271675051735381",NaN,NaN,Une femme,35-44 ans,"Actif, active (ayant un emploi, y compris en c...",...,Plutôt,Plutôt,Pas du tout,6.0,J'ai voté blanc ou nul,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,Internet
3344,5915.0,9hz0jyf2sw5ex73u,DOM TOM,CAWI Outre-mer,"0,440508231628487",NaN,NaN,Une femme,45-59 ans,"Actif, active (ayant un emploi, y compris en c...",...,Pas du tout,Tout à fait,Pas du tout,7.0,Marine Le Pen,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,Les fausses informations les pubs mensongères


Les réponses aux questions sont maintenant lisibles mais ce n'est pas fini, il reste à indiquer à quoi correspondent les noms de colonnes. On pourrait se dire qu'il suffit de remplacer ces noms par la question correspondante mais c'est un peu plus compliqué. Il existe 3 types de questions dans ce questionnaire

- Question simple à choix simple (Par exemple "Indiquez votre tranche d'âge"): Dans ce cas, une seule colonne correspond à la question et la réponse est donnée systématiquement.
- Question multiple à choix simple (Par exemple "Voici 5 affirmations, indiquez si vous êtes d'accord, pas d'accord,..."): Dans ce cas plusieurs colonnes correspondent à la même question et la réponse est donnée systématiquement.
- Question simple à choix multiple (Par exemple "Parmi ces 8 chaines de TV, lesquelles regardez-vous?"): Dans ce cas il y a autant de colonnes que de réponses possibles. La première colonne indique la première réponse donnée par l'interrogé, etc,.. Si l'interrogé a donné 3 réponses, les colonnes 1 à 3 les contiennent et les suivantes sont vides (NaN).

De plus, les questions pouvant être très longues, je préfère fournir dans un dictionnaire à part les énoncés des questions et les colonnes correspondantes, de manière à garder des noms de colonne simple et facile à utiliser comme "RS16_R_1".

In [ ]:
dico_questions = {
    "CIBLE": "Cible",
    "SOURCE_BASE_R1": "Source",
    "POIDS03": "Poids",
    "RS1_R": "Genre",
    "RS2C_RECODE_AG_R": "Age",
    "RS3_R":"Situation professionnelle",
    "RS4_R": "Profession",
    "CSPIND_R": "CSP",
    "RS5_R": "Niveau d'étude",
    "UDA9_R": "Région en 9",
    "UDA5_R": "Région en 5",
    "DPT_DOM": "DOM",
    "CC_R": "Taille agglo",
    "RS7_R":"Nombre de personnes dans le foyer",
    "RS14_R": "Revenu mensuel net du foyer",
    "ILLECTRO_R": "Dernière utilisation d'internet",
    "RS15_R": "Fréquence d'utilisation d'internet",
    "FREQ_INTERNET_R": "Fréquence d'utilisation d'internet (recode)",
    
    "RS16_R": "Équipement média",
    "RS16_R_1": "un poste de TV",
    "RS16_R_2": "un poste de radio, une chaine hifi",
    "RS16_R_3": "un autoradio",
    "RS16_R_4": "un smartphone",
    "RS16_R_5": "un ordinateur fixe",
    "RS16_R_6": "un ordianteur portable",
    "RS16_R_7": "une tablette tactile",
    "RS16_R_8": "la google chromecast, le firestick amazon ou une box appleTV",
    "RS16_R_9": "une enceinte connectée avec assistant vocal",
    "RS16_R_10": "un casque VR",
    "RS16_R_11": "une montre connectée",
    "RS16_R_12": "une console de jeux",
    
    "INT1_R": "Intérêt global pour l'information",
    
    "INT2_R": "Intérêt détaillé pour l'information sur un thème",
    "INT2_R_1": "Actualité internationale",
    "INT2_R_2": "Actualité nationale",
    "INT2_R_3": "Actualité locale / régionale ",
    "INT2_R_4": "Politique",
    "INT2_R_5": "Économie",
    "INT2_R_6": "Société (justice, emplois, fait-divers)",
    "INT2_R_7": "Environnement",
    "INT2_R_8": "Santé, mode de vie et bien-être",
    "INT2_R_9": "Culture et divertissement",
    "INT2_R_10": "Sport",
    "INT2_R_11": "People",
    "INT2_R_12": "Science et technologie",
    
    "VERSION_QUESTIO_R": "Version du questionnaire",
    
    "INT3_R": "Sentiment d'être bien informé sur les thèmes d'intérêt",
    "INT3_R": "Intérêt détaillé pour l'information sur un thème",
    "INT3_R_1": "Actualité internationale",
    "INT3_R_2": "Actualité nationale",
    "INT3_R_3": "Actualité locale / régionale ",
    "INT3_R_4": "Politique",
    "INT3_R_5": "Économie",
    "INT3_R_6": "Société (justice, emplois, fait-divers)",
    "INT3_R_7": "Environnement",
    "INT3_R_8": "Santé, mode de vie et bien-être",
    "INT3_R_9": "Culture et divertissement",
    "INT3_R_10": "Sport",
    "INT3_R_11": "People",
    "INT3_R_12": "Science et technologie",
    
    "MOTIV_R_1": "Comprendre ce qui m'entoure",
    "MOTIV_R_2": "Rester informé des grands événements",
    "MOTIV_R_3": "Me faire ma propre opinion",
    "MOTIV_R_4": "Prendre des décisions éclairées en tant que citoyen",
    "MOTIV_R_5": "Pouvoir en discuter et débattre avec mon entourage",
    "MOTIV_R_6": "Me divertir",
    "MOTIV_R_7": "Découvrir des tendances ou cultures",
    "MOTIV_R_8": "Satisfaire ma curiosité",
    "MOTIV_R_9": "Passer le temps",
    "MOTIV_R_10": "M'instruire, me cultiver",
    "MOTIV_R_11": "Progresser dans mon travail, mes études",
    "MOTIV_R_12": "Connaître d'autres avis que le mien",
    "MOTIV_R_14": "Par habitude",
    "MOTIV_R_15": "Aucune",
    
    "NEWS1_R_1": "En écoutant la radio ou des podcast",
    "NEWS1_R_1": "En regardant la TV (replay ou direct)",
    "NEWS1_R_1": "En lisant un journal ou magazine",
    "NEWS1_R_1": "En vous connectant directement sur un site internet",
    "NEWS1_R_1": "En utilisant un moteur de recherche",
    "NEWS1_R_1": "En utilisant un portail d'actualité",
    "NEWS1_R_1": "En consultant les réseaux sociaux",
    "NEWS1_R_1": "En consultant les plateformes vidéo en ligne",
    "NEWS1_1B_R": "Support d'écoute radio ou podcast",
    "NEWS1_2B_R": "Support pour regarder la TV",
    "NEWS1_3B_R": "Support de lecture de journeaux ou magazines",
    "NEWS1SOURCES_LR6_R": "Source des informations consultées via un moteur de recherche",
    "NEWS1SOURCES_LR7_R": "Source des informations consultées via un portail d'actualité",
    "NEWS1SOURCES_LR8_R": "Source des informations consultées via les réseaux sociaux",
    "NEWS1SOURCES_LR8_R": "Source des informations consultées via des vidéos en lignes",
    "NEWS1BIS_R": "Mode principal d'information",

    "PAY_R_1": "J'ai acheté des journaux ou magazines à l'unité en point de vente presse",
    "PAY_R_2": "J'ai acheté des articles de journaux ou magazines à l'unité en ligne",
    "PAY_R_3": "J'ai payé un abonnement à un ou plusieurs journaux ou magazines",
    "PAY_R_4": "J'ai payé un abonnement à un kiosque numérique",
    "PAY_R_5": "J'ai payé un abonnement à un site d'information en ligne",
    "PAY_R_6": "J'ai payé un abonnement à une newsletter",
    "PAY_R_7": "J'ai payé pour d'autres abonnements (Twitch, …)",

    "AGREG_R_1": "Flipboard",
    "AGREG_R_2": "Upday",
    "AGREG_R_3": "Google actualités",
    "AGREG_R_4": "Microsoft start",
    "AGREG_R_5": "Opéra news",
    "AGREG_R_6": "Orange actu",
    "AGREG_R_7": "Portail MSN",
    "AGREG_R_8": "Yahoo actualités",
    
    "NOTIF_R": "Activation des notification",
    
    "RSINFO_1_LR_R_1":"Utilise Facebook",
    "RSINFO_1_LR_R_2":"Utilise Instagram",
    "RSINFO_1_LR_R_3":"Utilise X",
    "RSINFO_1_LR_R_4":"Utilise TikTok",
    "RSINFO_1_LR_R_5":"Utilise Snapchat",
    "RSINFO_1_LR_R_6":"Utilise Linkedin",
    "RSINFO_1_LR_R_7":"Utilise WhatsApp",
    "RSINFO_1_LR_R_8":"Utilise Telegram",
    "RSINFO_1_LR_R_9":"Utilise Youtube",
    "RSINFO_1_LR_R_10":"Utilise Twitch",
    "RSINFO_1_LR_R_11":"Utilise Reddit",
    "RSINFO_1_LR_R_12":"Utilise Discord",
    "RSINFO_1_LR_R_13":"Utilise un autre réseau",
    "RSINFO_1_LR_R_14":"Aucun",
    
    "RSINFO_2_LR_R_1":"S'informe avec Facebook",
    "RSINFO_2_LR_R_2":"S'informe avec Instagram",
    "RSINFO_2_LR_R_3":"S'informe avec X",
    "RSINFO_2_LR_R_4":"S'informe avec TikTok",
    "RSINFO_2_LR_R_5":"S'informe avec Snapchat",
    "RSINFO_2_LR_R_6":"S'informe avec Linkedin",
    "RSINFO_2_LR_R_7":"S'informe avec WhatsApp",
    "RSINFO_2_LR_R_8":"S'informe avec Telegram",
    "RSINFO_2_LR_R_9":"S'informe avec Youtube",
    "RSINFO_2_LR_R_10":"S'informe avec Reddit",
    "RSINFO_2_LR_R_11":"S'informe avec Discord",
    "RSINFO_2_LR_R_12":"S'informe avec Twitch",
    "RSINFO_2_LR_R_13":"S'informe avec un autre réseau",
    }